In [5]:
import warnings
warnings.filterwarnings("ignore")

import yfinance as yf
import pandas as pd
import vectorbt as vbt
import pytz
import numpy as np

# ==============================
# CONFIG
# ==============================
symbol = "^NSEI"
interval = "1m"
period = "7d"

initial_sl_pct = 0.001     # 0.1%
trailing_sl_pct = 0.0001  # 0.03%

timezone = pytz.timezone("Asia/Kolkata")

# ==============================
# DOWNLOAD DATA
# ==============================
data = yf.download(
    tickers=symbol,
    interval=interval,
    period=period,
    progress=False
)

data.index = data.index.tz_convert(timezone)
data = data.between_time("09:15", "15:30")

close = data["Close"].squeeze()



In [6]:
# ==============================
# INDICATORS
# ==============================
ema7 = vbt.MA.run(close, window=7, ewm=True).ma
ema21 = vbt.MA.run(close, window=21, ewm=True).ma

# ==============================
# ENTRY SIGNAL
# ==============================
entries = ema7.vbt.crossed_above(ema21)

# ==============================
# CREATE TRADE ID (VERY IMPORTANT)
# ==============================
trade_id = entries.cumsum()
trade_id = trade_id.replace(0, np.nan)

# ==============================
# HIGHEST PRICE SINCE ENTRY
# ==============================
highest_price = close.groupby(trade_id).cummax()

# ==============================
# INITIAL STOP LOSS (0.1%)
# ==============================
initial_sl_price = close * (1 - initial_sl_pct)

# ==============================
# TRAILING STOP LOSS (0.03%)
# ==============================
trailing_sl_price = highest_price * (1 - trailing_sl_pct)

# ==============================
# FINAL STOP LOSS (NEVER MOVES DOWN)
# ==============================
final_sl = pd.concat(
    [initial_sl_price, trailing_sl_price],
    axis=1
).max(axis=1)

# ==============================
# STOP LOSS EXIT
# ==============================
sl_exit = close < final_sl

# ==============================
# TIME EXIT (3:15 PM)
# ==============================
time_exit = close.index.time >= pd.to_datetime("15:15").time()

# ==============================
# COMBINED EXITS
# ==============================
exits = sl_exit | time_exit

# ==============================
# PORTFOLIO
# ==============================
pf = vbt.Portfolio.from_signals(
    close=close,
    entries=entries,
    exits=exits,
    direction="longonly",
    freq="1T",
    init_cash=100000
)

# ==============================
# RESULTS
# ==============================
# print(pf.stats())
# pf.plot().show()

In [8]:

# # ==============================
# # RESULTS
# # ==============================
print(pf.stats())
# pf.plot().show()

Start                         2026-01-29 09:15:00+05:30
End                           2026-02-06 15:29:00+05:30
Period                                  1 days 13:24:00
Start Value                                    100000.0
End Value                                  100640.93974
Total Return [%]                                0.64094
Benchmark Return [%]                           1.621482
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                               0.376694
Max Drawdown Duration                   0 days 13:15:00
Total Trades                                         65
Total Closed Trades                                  65
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                  43.076923
Best Trade [%]                                  0.19026
Worst Trade [%]                               -0

In [9]:
trades_df = pf.trades.records_readable.copy()

for col in trades_df.columns:
    if pd.api.types.is_datetime64tz_dtype(trades_df[col]):
        trades_df[col] = trades_df[col].dt.tz_localize(None)

# trades_df.to_excel("nifty_ema_trades_without_fees_with_SL.xlsx", index=False)
trades_df.to_excel("Testing.xlsx", index=False)